In [8]:
import os
import json
import datetime as dt
import pandas as pd
import numpy as np
import sys
import requests
import datetime as dt

In [9]:
##get trailhead hotspot list

trailheadHotspots = pd.DataFrame()
trailheadHotspots = pd.read_csv('https://raw.githubusercontent.com/Tanag3r/trailheadDirect_birds/main/trailheadHotspots_Y2022_M1_D6.csv')

In [10]:
ebird_token = 'j6c7l80ga2ib'

##be nice to eBird, get a filtered list from all trailhead hotspots for testing
##the code below is for testing, interchange stopNames as needed
THfilters = ['HighSchool','EastSunsetWay']
THfiltersBool = trailheadHotspots.StopName.isin(THfilters)
selectHotspots = pd.DataFrame(trailheadHotspots[THfiltersBool])

In [11]:

##get recent notable observations

notableObs_current = []

##Really, really need to switch out this loop for a function
for locId in trailheadHotspots.itertuples():
##for locId in selectHotspots.itertuples():
    ebird_url = 'https://api.ebird.org/v2/data/obs/geo/recent/notable?'
    ebird_authHeader = {'X-eBirdApiToken': ebird_token}
    ebird_params = {
        'lat': str(locId.lat),
        'lng': str(locId.lng),
        'dist': str(2),
        'back': str(30),
        'detail': 'simple',
        'fmt': 'json'
    }
    ebird_request = requests.get(ebird_url,params=ebird_params,headers=ebird_authHeader)
    if ebird_request.status_code == requests.codes.ok:
        ebird_response = pd.DataFrame(ebird_request.json())
        ebird_response['StopName'] = locId.StopName
        notableObs_current.append(ebird_response)
    ebird_request.raise_for_status()

In [12]:
all_notableObs_current = pd.concat(notableObs_current)
##sanity check
all_notableObs_current.head(25)

,StopName,speciesCode,comName,sciName,locId,locName,obsDt,howMany,lat,lng,obsValid,obsReviewed,locationPrivate,subId
0,EastSunsetWay,comred,Common Redpoll,Acanthis flammea,L321969,Lake Sammamish SP,2022-01-01 07:37,1.0,47.559756,-122.054982,True,True,False,S99862395
0,EastSunsetWay,comred,Common Redpoll,Acanthis flammea,L321969,Lake Sammamish SP,2022-01-01 07:37,1.0,47.559756,-122.054982,True,True,False,S99862395
0,EastSunsetWay,comred,Common Redpoll,Acanthis flammea,L321969,Lake Sammamish SP,2022-01-01 07:37,1.0,47.559756,-122.054982,True,True,False,S99862395
0,HighSchool,comred,Common Redpoll,Acanthis flammea,L321969,Lake Sammamish SP,2022-01-01 07:37,1.0,47.559756,-122.054982,True,True,False,S99862395
0,HighSchool,comred,Common Redpoll,Acanthis flammea,L321969,Lake Sammamish SP,2022-01-01 07:37,1.0,47.559756,-122.054982,True,True,False,S99862395
0,HighSchool,comred,Common Redpoll,Acanthis flammea,L321969,Lake Sammamish SP,2022-01-01 07:37,1.0,47.559756,-122.054982,True,True,False,S99862395


In [13]:
##get historical data on all notable observations in a +/- 30 day window

date_range = []

today = dt.date.today()
todayBack = today - dt.timedelta(30)
todayOut = today + dt.timedelta(30)
yearStart = todayBack.year - 4
yearStop = todayOut.year - 1
monthBack = todayBack.month
monthOut = todayOut.month

for i in range(yearStart,yearStop):
    days = pd.date_range(
        start=dt.date(i,monthBack,todayBack.day),
        end=dt.date(i+1,monthOut,todayOut.day)
    )
    date_range.append(days)


In [14]:
date_list = []
for sublist in date_range:
    for item in sublist:
        date_list.append(item)

In [106]:
##get historic notable observations at the given hotspots
##TODO: #5 fix stopname loop problem --DONE

histNotableObs_atLoc = []
notable_locFrame = pd.DataFrame(all_notableObs_current)

for locId in notable_locFrame.itertuples():
    for date in date_list:
        ##time.sleep(0.5)
        ymd = '{}/{}/{}'.format(date.year,date.month,date.day)
        ebirdHist_url = 'https://api.ebird.org/v2/data/obs/' + locId.locId + '/historic/' + ymd
        ebirdHist_auth = {'X-eBirdApiToken': ebird_token}
        ebirdHist_params = {
            'fmt':'json',
            'detail':'simple'
        }
        ebirdHist_req = requests.get(ebirdHist_url,params=ebirdHist_params,headers=ebirdHist_auth)
        if ebirdHist_req.status_code == requests.codes.ok:
            ebirdHist_resp = pd.DataFrame(ebirdHist_req.json())
            ebirdHist_resp['StopName'] = locId.StopName
            histNotableObs_atLoc.append(ebirdHist_resp)
            ##ebirdHist_resp['StopName'] = pd.merge(ebirdHist_resp,all_notableObs_current[['StopName']],left_on='locId',right_on='locId',how='left')
        ebirdHist_req.raise_for_status()

In [107]:
all_histNotableObs = pd.concat(histNotableObs_atLoc)
##another sanity check
all_histNotableObs.head(15)

,speciesCode,comName,sciName,locId,locName,obsDt,howMany,lat,lng,obsValid,obsReviewed,locationPrivate,subId,StopName
0,virrai,Virginia Rail,Rallus limicola,L321969,Lake Sammamish SP,2017-12-11 16:12,1.0,47.559756,-122.054982,True,False,False,S41038268,EastSunsetWay
1,cacgoo1,Cackling Goose,Branta hutchinsii,L321969,Lake Sammamish SP,2017-12-11 13:15,400.0,47.559756,-122.054982,True,False,False,S41037213,EastSunsetWay
2,cangoo,Canada Goose,Branta canadensis,L321969,Lake Sammamish SP,2017-12-11 13:15,12.0,47.559756,-122.054982,True,False,False,S41037213,EastSunsetWay
3,norsho,Northern Shoveler,Spatula clypeata,L321969,Lake Sammamish SP,2017-12-11 13:15,1.0,47.559756,-122.054982,True,False,False,S41037213,EastSunsetWay
4,gadwal,Gadwall,Mareca strepera,L321969,Lake Sammamish SP,2017-12-11 13:15,2.0,47.559756,-122.054982,True,False,False,S41037213,EastSunsetWay
5,mallar3,Mallard,Anas platyrhynchos,L321969,Lake Sammamish SP,2017-12-11 13:15,25.0,47.559756,-122.054982,True,False,False,S41037213,EastSunsetWay
6,rinduc,Ring-necked Duck,Aythya collaris,L321969,Lake Sammamish SP,2017-12-11 13:15,3.0,47.559756,-122.054982,True,False,False,S41037213,EastSunsetWay
7,buffle,Bufflehead,Bucephala albeola,L321969,Lake Sammamish SP,2017-12-11 13:15,4.0,47.559756,-122.054982,True,False,False,S41037213,EastSunsetWay
8,hoomer,Hooded Merganser,Lophodytes cucullatus,L321969,Lake Sammamish SP,2017-12-11 13:15,5.0,47.559756,-122.054982,True,False,False,S41037213,EastSunsetWay
9,commer,Common Merganser,Mergus merganser,L321969,Lake Sammamish SP,2017-12-11 13:15,6.0,47.559756,-122.054982,True,False,False,S41037213,EastSunsetWay


In [108]:
##filter for notables
SpeciesFilters = list(all_notableObs_current['speciesCode'])
SpeciesFilters_bool = all_histNotableObs.speciesCode.isin(SpeciesFilters)
select_obs = pd.DataFrame(all_histNotableObs[SpeciesFilters_bool])

select_obs.head(25)

,speciesCode,comName,sciName,locId,locName,obsDt,howMany,lat,lng,obsValid,obsReviewed,locationPrivate,subId,StopName
31,comred,Common Redpoll,Acanthis flammea,L321969,Lake Sammamish SP,2017-12-11 13:15,8.0,47.559756,-122.054982,True,True,False,S41037213,EastSunsetWay
32,comred,Common Redpoll,Acanthis flammea,L321969,Lake Sammamish SP,2018-01-02 09:51,14.0,47.559756,-122.054982,True,True,False,S41557706,EastSunsetWay
35,comred,Common Redpoll,Acanthis flammea,L321969,Lake Sammamish SP,2018-02-05 09:37,12.0,47.559756,-122.054982,True,True,False,S42519977,EastSunsetWay
31,comred,Common Redpoll,Acanthis flammea,L321969,Lake Sammamish SP,2017-12-11 13:15,8.0,47.559756,-122.054982,True,True,False,S41037213,EastSunsetWay
32,comred,Common Redpoll,Acanthis flammea,L321969,Lake Sammamish SP,2018-01-02 09:51,14.0,47.559756,-122.054982,True,True,False,S41557706,EastSunsetWay
35,comred,Common Redpoll,Acanthis flammea,L321969,Lake Sammamish SP,2018-02-05 09:37,12.0,47.559756,-122.054982,True,True,False,S42519977,EastSunsetWay
31,comred,Common Redpoll,Acanthis flammea,L321969,Lake Sammamish SP,2017-12-11 13:15,8.0,47.559756,-122.054982,True,True,False,S41037213,EastSunsetWay
32,comred,Common Redpoll,Acanthis flammea,L321969,Lake Sammamish SP,2018-01-02 09:51,14.0,47.559756,-122.054982,True,True,False,S41557706,EastSunsetWay
35,comred,Common Redpoll,Acanthis flammea,L321969,Lake Sammamish SP,2018-02-05 09:37,12.0,47.559756,-122.054982,True,True,False,S42519977,EastSunsetWay
31,comred,Common Redpoll,Acanthis flammea,L321969,Lake Sammamish SP,2017-12-11 13:15,8.0,47.559756,-122.054982,True,True,False,S41037213,HighSchool


In [109]:
##clean df
select_obs['obsDt'] = select_obs['obsDt'].astype('datetime64[ns]')